In [ ]:
# script to create reaction database out of existing species_database and reaction_list.csv

# the problem is that we're currently using SMILES, and that's not actually unique and it's starting to cause problems

In [1]:
import os
import sys
import pandas as pd

import rmgpy.chemkin

sys.path.append('/work/westgroup/harris.se/autoscience/reaction_calculator/dft')
import importlib
import thermokinetic_fun
importlib.reload(thermokinetic_fun)

<module 'thermokinetic_fun' from '/work/westgroup/harris.se/autoscience/reaction_calculator/dft/thermokinetic_fun.py'>

In [6]:
# Load old reaction list
reaction_csv = os.path.join(thermokinetic_fun.DFT_DIR, 'reaction_list.csv')
reaction_df = pd.read_csv(reaction_csv)

species_df = pd.read_csv(os.path.join(thermokinetic_fun.DFT_DIR, 'species_list.csv'))

In [7]:
reaction_df

,Unnamed: 0,i,name,SMILES
0,0,0,H(14) + O2(2) <=> OH(15) + O(5),[H]+[O][O]_[OH]+[O]
1,1,1,H2(13) + O(5) <=> H(14) + OH(15),[H][H]+[O]_[H]+[OH]
2,2,2,H2(13) + OH(15) <=> H2O(8) + H(14),[H][H]+[OH]_O+[H]
3,3,3,OH(15) + OH(15) <=> H2O(8) + O(5),[OH]+[OH]_O+[O]
4,4,4,H2(13) <=> H(14) + H(14),[H][H]_[H]+[H]
...,...,...,...,...
4715,4715,4715,HO2(16) + CH2CHO(21) <=> O=O(690) + C2H3OH(58),[O]O+[CH2]C=O_O=O+C=CO
4716,4716,4716,O=O(690) + C2H3OH(58) <=> CO(6) + CH3O2H(46),O=O+C=CO_[C-]#[O+]+COO
4717,4717,4717,O=O(690) + C2H3OH(58) <=> HO2CH2CHO(147),O=O+C=CO_O=CCOO
4718,4718,4718,C2H3(22) + CH3CHO(35) <=> C2H3(22) + C[CH][O](...,[CH]=C+CC=O_[CH]=C+C[CH][O]


In [9]:
# new database will store name, SMILES, and some kind of unique identifier

# the species identifier will use indices from the species database, for example
# 23+928=39+929
# always in sorted order to tell if unique. Remember, species numbers will be unique from now on

# for initial construction, assume we can look species up by smiles
# since we haven't added anything that needs an adjacency list


entries = []
columns = ['i', 'name', 'SMILES', 'unique_string']

for i in range(len(reaction_df)):
    name = reaction_df['name'].values[i]
    smiles = reaction_df['SMILES'].values[i]

    # get the numbers
    rxn = thermokinetic_fun.smiles2reaction(smiles)
    reactants = []
    for sp in rxn.reactants:
        sp_index = species_df[species_df['SMILES'] == sp.smiles]['i'].values[0]
        reactants.append(sp_index)
    products = []
    for sp in rxn.products:
        sp_index = species_df[species_df['SMILES'] == sp.smiles]['i'].values[0]
        products.append(sp_index)
    reactants.sort()
    products.sort()

    reactants = [str(sp) for sp in reactants]
    products = [str(sp) for sp in products]
    unique_string = '+'.join(reactants) + '=' + '+'.join(products)
    
    # make sure the 'unique_string' is actually unique compared to everything that came before it
    already_exists = False
    for entry in entries:
        if entry[-1] == unique_string:
            already_exists = True
            break
    if already_exists:
        continue
    
    entries.append([i, name, smiles, unique_string])

reaction_database = pd.DataFrame(entries, columns=columns)



In [10]:
reaction_database

,i,name,SMILES,unique_string
0,0,H(14) + O2(2) <=> OH(15) + O(5),[H]+[O][O]_[OH]+[O],5+17=8+18
1,1,H2(13) + O(5) <=> H(14) + OH(15),[H][H]+[O]_[H]+[OH],8+16=17+18
2,2,H2(13) + OH(15) <=> H2O(8) + H(14),[H][H]+[OH]_O+[H],16+18=11+17
3,3,OH(15) + OH(15) <=> H2O(8) + O(5),[OH]+[OH]_O+[O],18+18=8+11
4,4,H2(13) <=> H(14) + H(14),[H][H]_[H]+[H],16=17+17
...,...,...,...,...
4684,4715,HO2(16) + CH2CHO(21) <=> O=O(690) + C2H3OH(58),[O]O+[CH2]C=O_O=O+C=CO,19+24=112+121
4685,4716,O=O(690) + C2H3OH(58) <=> CO(6) + CH3O2H(46),O=O+C=CO_[C-]#[O+]+COO,112+121=9+37
4686,4717,O=O(690) + C2H3OH(58) <=> HO2CH2CHO(147),O=O+C=CO_O=CCOO,112+121=188
4687,4718,C2H3(22) + CH3CHO(35) <=> C2H3(22) + C[CH][O](...,[CH]=C+CC=O_[CH]=C+C[CH][O],25+33=25+230


In [ ]:
# save results
reaction_database.to_csv(os.path.join(thermokinetic_fun.DFT_DIR, 'reaction_database.csv'), index=False)